In [3]:
from pycocotools.coco import COCO

In [4]:
import random

In [5]:
dataDir=<Path to coco2017 dataset>
dataType='train2017'
annFile='{}/annotations/instances_{}.json'.format(dataDir,dataType)

In [ ]:
coco=COCO(annFile)

In [40]:
# display COCO categories and supercategories
cats = coco.loadCats(coco.getCatIds())
nms=[cat['name'] for cat in cats]
print('COCO categories: \n{}\n'.format(' '.join(nms)))

nms = set([cat['supercategory'] for cat in cats])
print('COCO supercategories: \n{}'.format(' '.join(nms)))

COCO categories: 
person bicycle car motorcycle airplane bus train truck boat traffic light fire hydrant stop sign parking meter bench bird cat dog horse sheep cow elephant bear zebra giraffe backpack umbrella handbag tie suitcase frisbee skis snowboard sports ball kite baseball bat baseball glove skateboard surfboard tennis racket bottle wine glass cup fork knife spoon bowl banana apple sandwich orange broccoli carrot hot dog pizza donut cake chair couch potted plant bed dining table toilet tv laptop mouse remote keyboard cell phone microwave oven toaster sink refrigerator book clock vase scissors teddy bear hair drier toothbrush

COCO supercategories: 
vehicle sports electronic food accessory animal outdoor appliance person furniture kitchen indoor


In [41]:
cats[0]

{'id': 1, 'name': 'person', 'supercategory': 'person'}

In [42]:
# get all images containing given categories, select one at random
catIds = coco.getCatIds(catNms=['vehicle']);

In [43]:
imgIds = coco.getImgIds(catIds=catIds);

In [44]:
len(imgIds)

118287

In [45]:
cats_to_pack = []
cat_img = {}
cat_counter = 1 # leave zero for background class
for cat in cats:
    imgIds = coco.getImgIds(catIds=[cat['id']])
    if len(imgIds) > 4561:
        cats_to_pack.append(cat['name'])
        cat_img[cat['name']] = {"imgids":imgIds, "catid":cat_counter, "cat_ogid":cat['id']}
        cat_counter += 1
        print('{0} : {1}'.format(cat['name'], len(imgIds)))
cat_counter -= 1
print("Total Categories : {}".format(cat_counter))

person : 64115
car : 12251
truck : 6127
bench : 5570
backpack : 5528
handbag : 6841
bottle : 8501
cup : 9189
bowl : 7111
chair : 12774
dining table : 11837
cell phone : 4803
sink : 4678
book : 5332
clock : 4659
Total Categories : 15


In [46]:
#I wanted max object instances for any 15 categories so i choosed uppercap on number of instances of object
#you may have your own criteria
upper_cap = 4600

In [47]:
selected_imgs = {}
#randomly select upper cap images
#select random instances of object but kept similar number of objects per class to avoid overweighing of single class 
for cat_name, val in cat_img.items():
    img_ids = val["imgids"]
    print(len(img_ids))
    random.shuffle(img_ids)
    random.shuffle(img_ids)
    selected_imgs[cat_name] = {"imgids":img_ids[:upper_cap], "catid":val["catid"], "cat_ogid":val['cat_ogid']}

8501
6127
6841
4659
12251
4678
11837
64115
4803
5528
9189
5570
7111
5332
12774


# Create TFRecord

In [48]:
import tensorflow as tf

In [49]:
import PIL.Image

In [50]:
from tf_record_creation_util import open_sharded_output_tfrecords

In [51]:
import contextlib2

In [52]:
import os

In [53]:
# HELPER FUNCTIONS

def int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def int64_list_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))


def bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def bytes_list_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))


def float_list_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

In [54]:
class ImageCoder(object):
    """Helper class that provides TensorFlow image coding utilities."""

    def __init__(self):
        # Create a single Session to run all image coding calls.
        self._sess = tf.Session()

        # Initializes function that converts PNG to JPEG data.
        self._png_data = tf.placeholder(dtype=tf.string)
        image = tf.image.decode_png(self._png_data, channels=3)
        self._png_to_jpeg = tf.image.encode_jpeg(image, format='rgb', quality=100)

        # Initializes function that converts CMYK JPEG data to RGB JPEG data.
        self._cmyk_data = tf.placeholder(dtype=tf.string)
        image = tf.image.decode_jpeg(self._cmyk_data, channels=0)
        self._cmyk_to_rgb = tf.image.encode_jpeg(image, format='rgb', quality=100)

        # Initializes function that decodes RGB JPEG data.
        self._decode_jpeg_data = tf.placeholder(dtype=tf.string)
        self._decode_jpeg = tf.image.decode_jpeg(self._decode_jpeg_data, channels=3)

    def png_to_jpeg(self, image_data):
        return self._sess.run(self._png_to_jpeg,
                          feed_dict={self._png_data: image_data})

    def cmyk_to_rgb(self, image_data):
        return self._sess.run(self._cmyk_to_rgb,
                          feed_dict={self._cmyk_data: image_data})

    def decode_jpeg(self, image_data):
        image = self._sess.run(self._decode_jpeg,
                               feed_dict={self._decode_jpeg_data: image_data})
        assert len(image.shape) == 3
        assert image.shape[2] == 3
        return image

In [55]:
def _check_or_create_dir(directory):
    """Check if directory exists otherwise create it."""
    if not tf.gfile.Exists(directory):
        tf.gfile.MakeDirs(directory)

In [56]:
def create_tf_example(image,
                      annotations_list,
                      image_dir,
                      category_index,
                      include_masks=False):
    """Converts image and annotations to a tf.Example proto.
    Args:
    image: dict with keys:
      [u'license', u'file_name', u'coco_url', u'height', u'width',
      u'date_captured', u'flickr_url', u'id']
    annotations_list:
      list of dicts with keys:
      [u'segmentation', u'area', u'iscrowd', u'image_id',
      u'bbox', u'category_id', u'id']
      Notice that bounding box coordinates in the official COCO dataset are
      given as [x, y, width, height] tuples using absolute coordinates where
      x, y represent the top-left (0-indexed) corner.  This function converts
      to the format expected by the Tensorflow Object Detection API (which is
      which is [ymin, xmin, ymax, xmax] with coordinates normalized relative
      to image size).
    image_dir: directory containing the image files.
    category_index: a dict containing COCO category information keyed
      by the 'id' field of each category.  See the
      label_map_util.create_category_index function.
    include_masks: Whether to include instance segmentations masks
      (PNG encoded) in the result. default: False.
    Returns:
    example: The converted tf.Example
    num_annotations_skipped: Number of (invalid) annotations that were ignored.
    Raises:
    ValueError: if the image pointed to by data['filename'] is not a valid JPEG
    """
    image_height = image['height']
    image_width = image['width']
    filename = image['file_name']
    image_id = image['id']

    full_path = os.path.join(image_dir, filename)
    with tf.gfile.GFile(full_path, 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = PIL.Image.open(encoded_jpg_io)
    key = hashlib.sha256(encoded_jpg).hexdigest()

    xmin = []
    xmax = []
    ymin = []
    ymax = []
    is_crowd = []
    category_names = []
    category_ids = []
    area = []
    encoded_mask_png = []
    num_annotations_skipped = 0
    for object_annotations in annotations_list:
        (x, y, width, height) = tuple(object_annotations['bbox'])
        if width <= 0 or height <= 0:
            num_annotations_skipped += 1
            continue
        if x + width > image_width or y + height > image_height:
            num_annotations_skipped += 1
            continue
        xmin.append(float(x) / image_width)
        xmax.append(float(x + width) / image_width)
        ymin.append(float(y) / image_height)
        ymax.append(float(y + height) / image_height)
        is_crowd.append(object_annotations['iscrowd'])
        category_id = int(object_annotations['category_id'])
        category_ids.append(category_id)
        category_names.append(category_index[category_id]['name'].encode('utf8'))
        area.append(object_annotations['area'])

        if include_masks:
            run_len_encoding = mask.frPyObjects(object_annotations['segmentation'],
                                              image_height, image_width)
            binary_mask = mask.decode(run_len_encoding)
            if not object_annotations['iscrowd']:
                binary_mask = np.amax(binary_mask, axis=2)
            pil_image = PIL.Image.fromarray(binary_mask)
            output_io = io.BytesIO()
            pil_image.save(output_io, format='PNG')
            encoded_mask_png.append(output_io.getvalue())
            
    feature_dict = {
      'image/height':
          int64_feature(image_height),
      'image/width':
          int64_feature(image_width),
      'image/filename':
          bytes_feature(filename.encode('utf8')),
      'image/source_id':
          bytes_feature(str(image_id).encode('utf8')),
      'image/key/sha256':
          bytes_feature(key.encode('utf8')),
      'image/encoded':
          bytes_feature(encoded_jpg),
      'image/format':
          bytes_feature('jpeg'.encode('utf8')),
      'image/object/bbox/xmin':
          float_list_feature(xmin),
      'image/object/bbox/xmax':
          float_list_feature(xmax),
      'image/object/bbox/ymin':
          float_list_feature(ymin),
      'image/object/bbox/ymax':
          float_list_feature(ymax),
      'image/object/class/id':
          int64_list_feature(category_ids),
      'image/object/class/text':
          bytes_list_feature(category_names),
      'image/object/is_crowd':
          int64_list_feature(is_crowd),
      'image/object/area':
          float_list_feature(area),
    }
    
    if include_masks:
        feature_dict['image/object/mask'] = (
            bytes_list_feature(encoded_mask_png))
    example = tf.train.Example(features=tf.train.Features(feature=feature_dict))
    return key, example, num_annotations_skipped


In [57]:
tot_img_ids = []
tot_og_catids = []
tot_catids = []
id_lbl_map = {}
old_new_id_map = {}
category_index = {}
for cat_name, val in selected_imgs.items():
    tot_img_ids += val['imgids']
    tot_og_catids += [val['cat_ogid']]
    tot_catids += [val['catid']]
    id_lbl_map[val['catid']] = cat_name
    old_new_id_map[val['cat_ogid']] = val['catid']
    category_index[val['catid']] = {'id': val['catid'], 'name':cat_name}
    
tot_img_ids = list(set(tot_img_ids))
random.shuffle(tot_img_ids)
print(len(tot_img_ids))
print(tot_og_catids)
print(tot_catids)
print(old_new_id_map)
print(id_lbl_map)
print(category_index)

47705
[44, 8, 31, 3, 81, 67, 1, 77, 47, 27, 51, 15, 85, 84, 62]
[7, 3, 6, 2, 13, 11, 1, 12, 8, 5, 9, 4, 15, 14, 10]
{51: 9, 67: 11, 1: 1, 8: 3, 15: 4, 44: 7, 77: 12, 47: 8, 81: 13, 3: 2, 84: 14, 85: 15, 27: 5, 62: 10, 31: 6}
{1: 'person', 2: 'car', 3: 'truck', 4: 'bench', 5: 'backpack', 6: 'handbag', 7: 'bottle', 8: 'cup', 9: 'bowl', 10: 'chair', 11: 'dining table', 12: 'cell phone', 13: 'sink', 14: 'book', 15: 'clock'}
{1: {'id': 1, 'name': 'person'}, 2: {'id': 2, 'name': 'car'}, 3: {'id': 3, 'name': 'truck'}, 4: {'id': 4, 'name': 'bench'}, 5: {'id': 5, 'name': 'backpack'}, 6: {'id': 6, 'name': 'handbag'}, 7: {'id': 7, 'name': 'bottle'}, 8: {'id': 8, 'name': 'cup'}, 9: {'id': 9, 'name': 'bowl'}, 10: {'id': 10, 'name': 'chair'}, 11: {'id': 11, 'name': 'dining table'}, 12: {'id': 12, 'name': 'cell phone'}, 13: {'id': 13, 'name': 'sink'}, 14: {'id': 14, 'name': 'book'}, 15: {'id': 15, 'name': 'clock'}}


In [58]:
import sys
import pbtxthelper as pbh
import io
import hashlib

## Create Training tfrecords

In [24]:
pbtxt = 'labels.pbtxt'
output_dir = 'train_output'
num_of_shards  = 10
input_images_directory = os.path.join(dataDir, dataType)
print(input_images_directory)

#create labels pbtxt
pbh.dictToPbtxt(category_index, pbtxt)
base_pth = os.path.join(output_dir, dataType)

# sys.exit(1)
_check_or_create_dir(output_dir)
#Returns Tfrecords
with contextlib2.ExitStack() as tf_record_close_stack:
    output_tfrecords = open_sharded_output_tfrecords(
        tf_record_close_stack, output_dir,
        num_of_shards)
    
    for counter, img_id in enumerate(tot_img_ids):
        if (counter % 1000) == 0:
            print('Processed {} images...'.format(counter))
        
        img_annotations = coco.getAnnIds(imgIds=[img_id])
        img_annotations = coco.loadAnns(img_annotations)
#         print(img_annotations[0])
        new_anns = [x for x in img_annotations if x['category_id'] in tot_og_catids]
        for ann in new_anns:
#             print(ann)
            v = ann['category_id']
            ann['category_id'] = old_new_id_map[ann['category_id']]
#             print('changed {0} from {1}'.format(ann['category_id'], v))
            
        img_data = coco.loadImgs([img_id])
#         print(new_anns)
#         img_path = os.path.join(input_images_directory, img_data[0]['file_name'])
#         print(img_path)
#         print(os.path.isfile(img_path))
        
        
        key, tf_example, num_annotations_skipped = create_tf_example(image=img_data[0],
                                                                     annotations_list=new_anns,
                                                                     image_dir=input_images_directory,
                                                                     category_index=category_index
                                                                     )
        if tf_example:
            shard_idx = int(img_id) % num_of_shards
            output_tfrecords[shard_idx].write(tf_example.SerializeToString())

/media/kt-gpu2/My Passport/Datasets/coco2017/train2017
{'id': 1, 'name': 'person'}
{'id': 2, 'name': 'car'}
{'id': 3, 'name': 'truck'}
{'id': 4, 'name': 'bench'}
{'id': 5, 'name': 'backpack'}
{'id': 6, 'name': 'handbag'}
{'id': 7, 'name': 'bottle'}
{'id': 8, 'name': 'cup'}
{'id': 9, 'name': 'bowl'}
{'id': 10, 'name': 'chair'}
{'id': 11, 'name': 'dining table'}
{'id': 12, 'name': 'cell phone'}
{'id': 13, 'name': 'sink'}
{'id': 14, 'name': 'book'}
{'id': 15, 'name': 'clock'}
Written pbtxt file
Processed 0 images...
Processed 1000 images...
Processed 2000 images...
Processed 3000 images...
Processed 4000 images...
Processed 5000 images...
Processed 6000 images...
Processed 7000 images...
Processed 8000 images...
Processed 9000 images...
Processed 10000 images...
Processed 11000 images...
Processed 12000 images...
Processed 13000 images...
Processed 14000 images...
Processed 15000 images...
Processed 16000 images...
Processed 17000 images...
Processed 18000 images...
Processed 19000 images

## Create Validation tfrecords

In [59]:
dataType='val2017'
annFile='{}/annotations/instances_{}.json'.format(dataDir,dataType)
coco=COCO(annFile)

cats_to_pack = []
cat_img = {}
cat_counter = 1 # leave zero for background class
for cat in cats:
    if cat['id'] in tot_og_catids:
        imgIds = coco.getImgIds(catIds=[cat['id']])
        cats_to_pack.append(cat['name'])
        cat_img[cat['name']] = {"imgids":imgIds, "catid":cat_counter, "cat_ogid":cat['id']}
        cat_counter += 1
        print('{0} : {1}'.format(cat['name'], len(imgIds)))
cat_counter -= 1
print("Total Categories : {}".format(cat_counter))

selected_imgs = {}
#randomly select upper cap images
for cat_name, val in cat_img.items():
    img_ids = val["imgids"]
    print(len(img_ids))
    random.shuffle(img_ids)
    random.shuffle(img_ids)
    selected_imgs[cat_name] = {"imgids":img_ids, "catid":val["catid"], "cat_ogid":val['cat_ogid']}
    
tot_img_ids = []
for cat_name, val in selected_imgs.items():
    tot_img_ids += val['imgids']

loading annotations into memory...
Done (t=0.48s)
creating index...
index created!
person : 2693
car : 535
truck : 250
bench : 235
backpack : 228
handbag : 292
bottle : 379
cup : 390
bowl : 314
chair : 580
dining table : 501
cell phone : 214
sink : 187
book : 230
clock : 204
Total Categories : 15
379
250
292
204
535
187
501
2693
214
228
390
235
314
230
580


In [60]:
#val tfrecord
pbtxt = 'labels.pbtxt'
output_dir = 'val_output'
num_of_shards  = 10
input_images_directory = os.path.join(dataDir, dataType)
print(input_images_directory)

base_pth = os.path.join(output_dir, dataType)

# sys.exit(1)
_check_or_create_dir(output_dir)
#Returns Tfrecords
with contextlib2.ExitStack() as tf_record_close_stack:
    output_tfrecords = open_sharded_output_tfrecords(
        tf_record_close_stack, output_dir,
        num_of_shards)
    
    for counter, img_id in enumerate(tot_img_ids):
        if (counter % 1000) == 0:
            print('Processed {} images...'.format(counter))
        
        img_annotations = coco.getAnnIds(imgIds=[img_id])
        img_annotations = coco.loadAnns(img_annotations)
#         print(img_annotations[0])
        new_anns = [x for x in img_annotations if x['category_id'] in tot_og_catids]
        for ann in new_anns:
#             print(ann)
            v = ann['category_id']
            ann['category_id'] = old_new_id_map[ann['category_id']]
#             print('changed {0} from {1}'.format(ann['category_id'], v))
            
        img_data = coco.loadImgs([img_id])
#         print(new_anns)
#         img_path = os.path.join(input_images_directory, img_data[0]['file_name'])
#         print(img_path)
#         print(os.path.isfile(img_path))
        
        
        key, tf_example, num_annotations_skipped = create_tf_example(image=img_data[0],
                                                                     annotations_list=new_anns,
                                                                     image_dir=input_images_directory,
                                                                     category_index=category_index
                                                                     )
        if tf_example:
            shard_idx = int(img_id) % num_of_shards
            output_tfrecords[shard_idx].write(tf_example.SerializeToString())

/media/kt-gpu2/My Passport/Datasets/coco2017/val2017
{'id': 1, 'name': 'person'}
{'id': 2, 'name': 'car'}
{'id': 3, 'name': 'truck'}
{'id': 4, 'name': 'bench'}
{'id': 5, 'name': 'backpack'}
{'id': 6, 'name': 'handbag'}
{'id': 7, 'name': 'bottle'}
{'id': 8, 'name': 'cup'}
{'id': 9, 'name': 'bowl'}
{'id': 10, 'name': 'chair'}
{'id': 11, 'name': 'dining table'}
{'id': 12, 'name': 'cell phone'}
{'id': 13, 'name': 'sink'}
{'id': 14, 'name': 'book'}
{'id': 15, 'name': 'clock'}
Written pbtxt file
Processed 0 images...
Processed 1000 images...
Processed 2000 images...
Processed 3000 images...
Processed 4000 images...
Processed 5000 images...
Processed 6000 images...
Processed 7000 images...
